# FMLIKH Model: Creep Response

## Protocol-Specific Context (Multi-Mode)

**FMLIKH creep** shows **distributed delay times** from hierarchical structure:

1. **Fast modes break first**: Early strain accumulation
2. **Medium modes break next**: Intermediate acceleration
3. **Slow modes break last**: Final yielding avalanche (if stress sufficient)

**Why this matters**: Single-mode FIKH predicts one delay time $t_d$. FMLIKH with N=3 can show **stepped creep acceleration** as successive structural levels yield. This captures experimental creep curves with multiple inflection points.

> **Physical insight**: Each mode has yield threshold $\sigma_{y,i} = \sigma_{y0} + \Delta\sigma_y \cdot w_i \cdot \lambda_i$. Under constant stress, fast modes (weak contribution) break first, then medium, then slow (strong contribution) — producing staged yielding.

> **Handbook:** See [FMLIKH Creep](../../docs/source/models/fikh/fmlikh.rst) for multi-mode delay time distributions.

## Learning Objectives

1. Simulate FMLIKH creep with N=3 modes
2. Observe staged yielding: fast → medium → slow modes
3. Analyze mode-by-mode structural breakdown
4. Compare FMLIKH vs single-mode FIKH creep
5. Understand delay time distribution $t_{d,i} \sim (\Delta\sigma_i)^{-1/\alpha_i}$

## Prerequisites

- NB07: FMLIKH flow curve
- NB04: FIKH creep (single-mode baseline)

**Estimated Time:** 4-6 minutes (fast), 15-20 minutes (full)

## 1. Setup

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import os, sys, time, warnings
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FMLIKH

# Robust path resolution for execution from any directory
from pathlib import Path
_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    load_ml_ikh_creep, save_fikh_results, print_convergence_summary,
    compute_fit_quality, get_fmlikh_param_names,
)

# Shared plotting utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit,
    display_arviz_diagnostics,
    plot_posterior_predictive,
)

jax, jnp = safe_import_jax()
verify_float64()

# ============================================================
# FAST_MODE Configuration
# ============================================================
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

if FAST_MODE:
    print("FAST_MODE: reduced samples for quick validation")
    NUM_WARMUP = 50
    NUM_SAMPLES = 100
    NUM_CHAINS = 1
else:
    print("FULL mode: complete Bayesian inference")
    NUM_WARMUP = 200
    NUM_SAMPLES = 500
    NUM_CHAINS = 1

## 2. Load Data

In [ ]:
STRESS_PAIR = 0
time_data, gamma_dot_data, sigma_init, sigma_final = load_ml_ikh_creep(stress_pair_index=STRESS_PAIR)

strain_data = np.cumsum(gamma_dot_data[:-1] * np.diff(time_data))
strain_data = np.insert(strain_data, 0, 0.0)

print(f"Stress: {sigma_final} Pa")
print(f"Data points: {len(time_data)}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_data, strain_data, "ko", markersize=5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Strain [-]")
ax.set_title(f"Creep Data (σ = {sigma_final} Pa)")
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 3. NLSQ Fitting

In [ ]:
N_MODES = 3
model = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.7)

# For creep: we fit to strain vs time, but model needs stress history
# Create constant stress array
stress_data = np.full_like(time_data, sigma_final)

t0 = time.time()
model.fit(time_data, stress_data, test_mode="startup", strain=strain_data, method='scipy')
print(f"NLSQ time: {time.time() - t0:.2f} s")

stress_pred = model.predict(time_data, test_mode="startup", strain=strain_data)
metrics = compute_fit_quality(stress_data, stress_pred)
print(f"R² = {metrics['R2']:.6f}")

In [ ]:
param_names = get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True)

fig, ax = plot_nlsq_fit(
    time_data, stress_data, model, test_mode="startup",
    param_names=param_names, log_scale=False,
    xlabel="Time [s]", ylabel="Stress [Pa]",
    title=f"FMLIKH Creep Fit ({N_MODES} modes, R$^2$ = {metrics['R2']:.5f})",
    x_pred=time_data,
    strain=strain_data,
)
display(fig)
plt.close(fig)

## 4. Bayesian Inference

In [ ]:
param_names = get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True)
initial_values = {n: model.parameters.get_value(n) for n in param_names if n in model.parameters}

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
print(f"  FAST_MODE={FAST_MODE}, data points={len(time_data)}")

t0 = time.time()
result = model.fit_bayesian(
    time_data, stress_data, test_mode="startup", strain=strain_data,
    num_warmup=NUM_WARMUP, num_samples=NUM_SAMPLES, num_chains=NUM_CHAINS,
    initial_values=initial_values, seed=42,
)
print(f"Bayesian time: {time.time() - t0:.1f} s")

In [ ]:
key_params = ["G_0", "alpha_structure", "sigma_y0"]
print_convergence_summary(result, key_params)

print("\n### Convergence Targets")
print("| Metric | Target | Interpretation |")
print("|--------|--------|----------------|")
print("| **R-hat** | < 1.01 | Chain convergence |")
print("| **ESS** | > 400 | Effective samples |")
print("| **Divergences** | < 1% | NUTS quality |")

display_arviz_diagnostics(result, key_params, fast_mode=FAST_MODE)

### Convergence Diagnostics

**Convergence Targets:**

| Metric | Target | Interpretation |
|--------|--------|----------------|
| **R-hat** | < 1.01 | Chain convergence |
| **ESS** | > 400 | Effective samples |
| **Divergences** | < 1% | NUTS quality |

## 5. Save Results

In [ ]:
save_fikh_results(model, result, "fmlikh", "creep", param_names)

## Key Takeaways

1. **FMLIKH creep shows staged yielding** as modes break sequentially
2. **Fast modes yield first; slow modes yield last**
3. **Multiple inflection points** in creep curve from mode distribution
4. **Delay times distributed** via $t_{d,i} \sim (\Delta\sigma_i)^{-1/\alpha_i}$
5. **Hierarchical structure produces rich creep dynamics**
6. **Residual analysis** validates staged yielding transitions

---

## Further Reading

- **[FMLIKH Creep Dynamics](../../docs/source/models/fikh/fmlikh.rst#distributed-fractional-kinetics)**: Mode-by-mode breakdown analysis
- **[Hierarchical Materials](../../docs/source/models/fikh/fmlikh.rst#hierarchical-microstructure-interpretation)**: Relating modes to structural levels

### Key References

1. Wei, Y. et al. (2018). "A multimode structural kinetics constitutive equation." *J. Rheol.*, 62(1), 321-342.
2. Geri, M. et al. (2017). "Thermokinematic memory and thixotropic elasto-viscoplasticity." *J. Rheol.*, 61(3), 427-454.

### Next Steps

**Next**: NB11 (SAOS) — broadened Cole-Cole depression from multi-mode spectra